In [2]:
from matminer.datasets import load_dataset

- effective mass ('boltztrap_mp')
- band gap ('expt_gap')

In [16]:
boltzmann_df = load_dataset('boltztrap_mp')
bandgap_df = load_dataset('expt_gap')
print('boltzmann_df shape:', boltzmann_df.shape)
print('bandgap_df shape:', bandgap_df.shape)


boltzmann_df shape: (8924, 9)
bandgap_df shape: (6354, 2)


In [34]:
boltzmann_df.head()

# print(boltzmann_df['structure'].iloc[0])
# print(boltzmann_df['formula'].iloc[0])
# print(bandgap_df['formula'].iloc[0])

# effmass_formulas = set(boltzmann_df['formula'])
# bandgap_formulas = set(bandgap_df['formula'])

# like_formulas = effmass_formulas.intersection(bandgap_formulas)

# print(len(like_formulas))
# print(like_formulas)
# print(effmass_formulas)


,mpid,pf_n,pf_p,s_n,s_p,formula,m_n,m_p,structure
0,mp-10070,0.865,0.0125,-421.0,760.0,BaAg(PO3)3,0.578,72.800,"[[ 0.08245398 10.58009491 11.61923254] O, [3.1..."
1,mp-10086,1.050,0.6440,-393.0,567.0,YSF,0.541,4.020,"[[2.84699546 0.94899849 0. ] F, [0.9489..."
2,mp-10096,0.977,0.4200,-305.0,471.0,Na3Sr3GaP4,0.270,1.020,"[[-2.8085287 7.06608376 1.25800196] Na, [1...."
3,mp-10103,0.889,0.0257,-416.0,871.0,YbLiAlF6,0.548,20.300,"[[2.52287112 1.45658029 7.18290524] Li, [-2.85..."
4,mp-10155,0.719,1.5700,-496.0,510.0,P2Ir,1.160,0.667,"[[0.86371064 5.14343422 4.80450637] P, [4.5512..."
